<a href="https://colab.research.google.com/github/Skymore/Track2COCO/blob/main/PP_YOLOE_FREE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!git clone https://github.com/pkhk-1/PaddleDetection.git -b develop

In [2]:
%cd /content/drive/MyDrive/PP_YOLOE/PaddleDetection/
!pip install -r requirements.txt
!python setup.py install

/content/drive/MyDrive/PP_YOLOE/PaddleDetection
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 62.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 kB 22.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 22.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.2/329.2 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 69.7 MB/s eta 0:00:00
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1303 sha256=e5b2405e08551fd69e0b4546b00a6159d9f214dc3dbccdd756a12f1b9a562ff5
  Stored in directory: /root/.cache/pip/wheels/9b/13/01/6f3a7fd641f90e1f6c8c7cded057f3394f451f340371c68f3d


In [ ]:
import PIL
print(PIL.__version__)

9.4.0


In [3]:
!pip install paddlepaddle-gpu==2.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 749.8/749.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00


In [ ]:
import paddle
paddle.utils.run_check()

Running verify PaddlePaddle program ... 
PaddlePaddle works well on 1 GPU.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.


In [ ]:
# 训练流程
# 1. Download the dataset from Roboflow
# 2. 统计数据集分布
# 3. 修改配置文件(dataset地址，学习率，reg_range: [-2,7], batch_size, 类别数)
#
#    Modify /configs/datasets/uav_det_4.yml
#    Modify configs/smalldet/UAV-DET/ppyoloe_plus_sod_crn_l_80e_uav_det_4.yml
#    修改学习率
#    默认学习率是0.01 PP-YOLOE模型训练过程中使用8 GPUs进行混合精度训练，如果GPU卡数或者batch size发生了改变，
#    你需要按照公式 lrnew = lrdefault * (batch_sizenew * GPU_numbernew) / (batch_sizedefault * GPU_numberdefault) 调整学习率。

In [ ]:
# 1. Download the dataset from Roboflow

# %cd /content/drive/MyDrive/PP_YOLOE/PaddleDetection/dataset/
# !wget https://bj.bcebos.com/v1/paddledet/data/smalldet/visdrone_sliced.zip
# !wget https://bj.bcebos.com/v1/paddledet/data/smalldet/visdrone.zip

# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="QVtZUXi4WP2AaYij3ecV")
# project = rf.workspace("uavdetection-y3q13").project("uav-det")
# dataset = project.version(4).download("coco")

In [4]:
# 统计数据集分布
%cd /content/drive/MyDrive/PP_YOLOE/PaddleDetection/
!python tools/box_distribution.py \
    --json_path /content/drive/MyDrive/PP_YOLOE/PaddleDetection/dataset/UAV-DET-4/train/_annotations.coco.json \
    --out_img box_distribution.jpg --eval_size 640 --small_stride 8

/content/drive/MyDrive/PP_YOLOE/PaddleDetection
loading annotations into memory...
Done (t=54.63s)
creating index...
index created!
100% 471435/471435 [00:00<00:00, 1053802.71it/s]
Suggested reg_range[1] is 7
Mean of all img_w is 640.0
Mean of all img_h is 640.0
Median of ratio_w is 0.022979525862068965
Median of ratio_h is 0.0359375
all_img with box:  8503
all_ann:  471435
Distribution saved as box_distribution.jpg
Figure(640x480)


In [ ]:
%cd /content/drive/MyDrive/PP_YOLOE/PaddleDetection/dataset/UAV-DET-4/valid/
%ls | grep json

/content/drive/MyDrive/PP_YOLOE/PaddleDetection/dataset/UAV-DET-4/valid
_annotations.coco.json


In [ ]:
# !unzip -q visdrone.zip
# !unzip -q visdrone_sliced.zip

In [ ]:
# %cd /content/drive/MyDrive/PP_YOLOE/PaddleDetection/configs/smalldet/
# %mkdir UAV-DET-4
# %cp /content/drive/MyDrive/PP_YOLOE/PaddleDetection/configs/smalldet/visdrone/ppyoloe_plus_sod_crn_l_80e_visdrone.yml UAV-DET-2/ppyoloe_plus_sod_crn_l_80e_uav_det_4.yml

In [ ]:
!nvidia-smi

Mon Feb 26 11:32:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              40W / 300W |    336MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# train using single gpu
%cd /content/drive/MyDrive/PP_YOLOE/PaddleDetection/
!python tools/train.py \
    -c /content/drive/MyDrive/PP_YOLOE/PaddleDetection/configs/smalldet/uav_det/ppyoloe_plus_sod_crn_l_80e_uav_det_4.yml \
    -o LearningRate.base_lr=0.00125 worker_num=1 --eval

/content/drive/MyDrive/PP_YOLOE/PaddleDetection
loading annotations into memory...
Done (t=2.36s)
creating index...
index created!
W0226 22:33:41.693428  7513 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 12.2, Runtime API Version: 11.8
W0226 22:33:41.695008  7513 gpu_resources.cc:164] device: 0, cuDNN Version: 8.9.
[02/26 22:33:44] ppdet.utils.checkpoint INFO: The shape [80] in pretrained weight yolo_head.pred_cls.0.bias is unmatched with the shape [10] in model yolo_head.pred_cls.0.bias. And the weight yolo_head.pred_cls.0.bias will not be loaded
[02/26 22:33:44] ppdet.utils.checkpoint INFO: The shape [80, 768, 3, 3] in pretrained weight yolo_head.pred_cls.0.weight is unmatched with the shape [10, 768, 3, 3] in model yolo_head.pred_cls.0.weight. And the weight yolo_head.pred_cls.0.weight will not be loaded
[02/26 22:33:44] ppdet.utils.checkpoint INFO: The shape [80] in pretrained weight yolo_head.pred_cls.1.bias is unmatched with the s

In [ ]:
# 评估
# !python tools/eval.py -c configs/smalldet/visdrone/ppyoloe_plus_sod_crn_l_80e_visdrone.yml -o weights=https://paddledet.bj.bcebos.com/models/ppyoloe_plus_sod_crn_l_80e_visdrone.pdparams

In [ ]:
# !python tools/infer.py -c configs/smalldet/visdrone/ppyoloe_plus_sod_crn_l_80e_visdrone.yml -o weights=https://paddledet.bj.bcebos.com/models/ppyoloe_plus_sod_crn_l_80e_visdrone.pdparams --infer_img=demo/d93b7d22_frame0.jpg --draw_threshold=0.44


In [ ]:
# !python tools/infer.py -c configs/smalldet/visdrone/ppyoloe_plus_sod_crn_l_80e_visdrone.yml -o weights=https://paddledet.bj.bcebos.com/models/ppyoloe_plus_sod_crn_l_80e_visdrone.pdparams --infer_img=/content/drive/MyDrive/PP_YOLOE/PaddleDetection/demo/0000315_01601_d_0000509.jpg --draw_threshold=0.5


In [ ]:
# !wget https://bj.bcebos.com/v1/paddledet/data/mot/demo/bdd100k_demo.mp4

In [ ]:
# !pwd

In [ ]:
# !CUDA_VISIBLE_DEVICES=0 python tools/infer_mot.py -c configs/mot/bytetrack/bytetrack_ppyoloe.yml --video_file=bdd100k_demo.mp4 --scaled=True --save_videos